In [1]:
import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import FullBatchLinkGenerator
from stellargraph.layer import GCN, LinkEmbedding
import networkx as nx

from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection

from stellargraph import globalvar,StellarGraph
from stellargraph import datasets
from IPython.display import display, HTML
import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, HinSAGE, link_classification

from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection

from stellargraph import globalvar
from stellargraph import datasets
from IPython.display import display, HTML
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
g1 = nx.read_graphml("2019-12-31.graphml")
g2 = nx.read_graphml("2020-01-01.graphml")
g3 = nx.read_graphml("2020-01-02.graphml")
g4 = nx.read_graphml("2020-01-03.graphml")
g5 = nx.read_graphml("2020-01-04.graphml")
g6 = nx.read_graphml("2020-01-05.graphml")
g7 = nx.read_graphml("2020-01-06.graphml")
g8 = nx.read_graphml("2020-01-07.graphml")


In [3]:
glist = [g2,g3,g4,g5,g6,g7,g8]
idx = len(glist)


In [4]:
G = nx.Graph()
while idx >= 0:
    idx -= 1
    G = nx.compose(G, glist[idx])
    attr_n_tx = {e: G.edges[e]['n_tx'] + glist[idx].edges[e]['n_tx'] for e in G.edges & glist[idx].edges}
    nx.set_edge_attributes(G, attr_n_tx, 'n_tx')
    attr_value = {e: G.edges[e]['value'] + glist[idx].edges[e]['value'] for e in G.edges & glist[idx].edges}
    nx.set_edge_attributes(G, attr_value, 'value')
    

In [5]:
def max_min_normalization(G):
    n_tx = nx.get_edge_attributes(G, "n_tx")
    value = nx.get_edge_attributes(G, "value")
    weight = {}
    n_tx_min = min(n_tx.values())
    n_tx_max = max(n_tx.values())
    value_min = min(value.values())
    value_max = max(value.values())
    for key in n_tx.keys():
        n_tx[key] = (n_tx[key] - n_tx_min)/(n_tx_max - n_tx_min)
    for key in value.keys():
        value[key] = (value[key] - value_min)/(value_max - value_min)
    for key in value.keys():
        weight[key] = n_tx[key] * 0.5 + value[key] * 0.5
    weight_min = min(weight.values())
    weight_max = max(weight.values())
    for key in weight.keys():
        weight[key] = (weight[key] - weight_min)/(weight_max - weight_min)
    nx.set_edge_attributes(G, weight, name="weight")
    return weight

In [6]:
#G1 = g1.copy()
G2 = G.copy()

In [7]:
#G1.remove_nodes_from(list(n for n in G1 if n not in G2))
G2.remove_nodes_from(list(n for n in G2 if n not in g1))

In [8]:
G.remove_nodes_from(list(n for n in G if n not in g1))
G.remove_edges_from(list(n for n in G.edges if n not in g1.edges))

In [9]:
#df_nodes2 = nx.to_pandas_adjacency(G)


In [10]:
G2.number_of_nodes()

88310

In [11]:
G2.number_of_edges()

133879

In [12]:
G.number_of_nodes()

88310

In [13]:
G.number_of_edges()

23726

In [14]:
max_min_normalization(G2)


{('89504130', '90418301'): 0.004246733843451866,
 ('89504130', '33610870'): 0.0026671667633686677,
 ('89504130', '23670066'): 0.009619716367799533,
 ('89504130', '18207386'): 0.0031493390949398074,
 ('89504130', '14756965'): 6.446309484042637e-07,
 ('89504130', '20785726'): 4.265812253205032e-06,
 ('89504130', '20870509'): 0.0022986497135519435,
 ('89504130', '309476372'): 2.168554186915605e-07,
 ('89504130', '15420689'): 5.649625538544507e-06,
 ('89504130', '136832'): 0.0005414131614541862,
 ('89504130', '221'): 1.878354949593038e-05,
 ('89504130', '426221227'): 5.454108990918666e-07,
 ('89504130', '382076042'): 6.199673966217471e-06,
 ('89504130', '49745408'): 0.0010517212833804019,
 ('89504130', '93700101'): 0.0026929276070220046,
 ('89504130', '435598288'): 7.266848604615233e-06,
 ('89504130', '52693139'): 3.468608649708321e-06,
 ('89504130', '50263989'): 0.004206905983665201,
 ('89504130', '196197096'): 3.9467613018977204e-06,
 ('89504130', '41431405'): 0.003217425216926017,
 ('89

In [15]:
max_min_normalization(G)

{('89504130', '90418301'): 0.004524501458924305,
 ('89504130', '23670066'): 0.010582922490079074,
 ('89504130', '18207386'): 0.0031705889682450646,
 ('89504130', '20785726'): 2.6612539457294514e-05,
 ('89504130', '309476372'): 1.352866238264436e-06,
 ('89504130', '221'): 0.00011718236117164477,
 ('89504130', '49745408'): 0.001068975690616919,
 ('89504130', '50263989'): 0.004276032837414439,
 ('89504130', '196197096'): 2.462211992465167e-05,
 ('89504130', '41431405'): 0.0035953485451965263,
 ('89504130', '15067129'): 0.030771469832507368,
 ('89504130', '96780285'): 0.0017833728066129703,
 ('89504130', '89157072'): 0.0021246867267233254,
 ('89504130', '43952585'): 0.024426641699136824,
 ('89504130', '21056449'): 0.002209327811542219,
 ('89504130', '289943900'): 7.548000879923687e-06,
 ('89504130', '454520125'): 1.0986620153070684e-06,
 ('89504130', '34515'): 0.0010574623434194937,
 ('89504130', '43858667'): 1.1388338186616317e-05,
 ('89504130', '14093991'): 0.0030857913797196993,
 ('8950

In [ ]:
dc1 = nx.degree_centrality(G)
dc2 = nx.degree_centrality(G2)
ec1 = nx.eigenvector_centrality(G,weight='weight')
ec2 = nx.eigenvector_centrality(G2,weight='weight')
cc1 = nx.closeness_centrality(G)
cc2 = nx.closeness_centrality(G2)

In [ ]:
#df_nodes1 = nx.to_pandas_adjacency(G2)
def compute_features1(node_id):
    # in general this could compute something based on other features, but for this example,
    # we don't have any other features, so we'll just do something basic with the node_id
    return [dc1[node_id],ec1[node_id],cc1[node_id]]

def compute_features2(node_id):
    # in general this could compute something based on other features, but for this example,
    # we don't have any other features, so we'll just do something basic with the node_id
    return [dc2[node_id],ec2[node_id],cc2[node_id]]

for node_id, node_data in G.nodes(data=True):
    node_data["feature"] = compute_features1(node_id)

for node_id, node_data in G2.nodes(data=True):
    node_data["feature"] = compute_features2(node_id)

In [ ]:
Gs1 = StellarGraph.from_networkx(
    G,node_features="feature"
)
Gs2 = StellarGraph.from_networkx(
    G2,node_features="feature"
)

In [ ]:
print(Gs1.info())

In [ ]:
print(Gs2.info())

In [ ]:
# Define an edge splitter on the original graph G:
edge_splitter_test = EdgeSplitter(Gs2)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G, and obtain the
# reduced graph G_test with the sampled links removed:
G_test, edge_ids_test, edge_labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global", keep_connected=False
)

In [ ]:
# Define an edge splitter on the reduced graph G_test:
edge_splitter_train = EdgeSplitter(Gs1)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G_test, and obtain the
# reduced graph G_train with the sampled links removed:
G_train, edge_ids_train, edge_labels_train = edge_splitter_train.train_test_split(
    p=0.1, method="global", keep_connected=False
)

In [ ]:
batch_size = 10
epochs = 50
num_samples = [20, 10]

In [ ]:
train_gen = GraphSAGELinkGenerator(G_train, batch_size, num_samples)
train_flow = train_gen.flow(edge_ids_train, edge_labels_train, shuffle=True)

In [ ]:
test_gen = GraphSAGELinkGenerator(G_test, batch_size, num_samples)
test_flow = test_gen.flow(edge_ids_test, edge_labels_test)

In [ ]:
layer_sizes = [20, 20]
graphsage = GraphSAGE(
    layer_sizes=layer_sizes, generator=train_gen, bias=True, dropout=0.3
)

In [ ]:
# Build the model and expose input and output sockets of graphsage model
# for link prediction
x_inp, x_out = graphsage.in_out_tensors()

In [ ]:
prediction = link_classification(
    output_dim=1, output_act="relu", edge_embedding_method="ip"
)(x_out)

In [ ]:
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=["acc"],
)

In [ ]:
init_train_metrics = model.evaluate(train_flow)
init_test_metrics = model.evaluate(test_flow)

print("\nTrain Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nTest Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

In [ ]:
history = model.fit(train_flow, epochs=epochs, validation_data=test_flow, verbose=2)

In [ ]:
sg.utils.plot_history(history)

In [ ]:
train_metrics = model.evaluate(train_flow)
test_metrics = model.evaluate(test_flow)

print("\nTrain Set Metrics of the trained model:")
for name, val in zip(model.metrics_names, train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nTest Set Metrics of the trained model:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))